In [4]:
import json
import random
from dataclasses import dataclass
from typing import List, Dict, Optional
from datetime import datetime, timedelta
import os

# Assume CoursesTYG.json is structured like this:
# {
#   "course": {
#     "metadata": { ... },
#     "categories": [ { "id": "cat1", ... } ],
#     "modules": [ { "id": "mod1", "title": "Module 1", "difficulty": {...}, "requiredEloRating": 100, "rubric": {"checklistItems": [...] } } ]
#   }
# }
# You'll need to create a sample CoursesTYG.json file for this to run.

@dataclass
class MockUser:
    id: str
    name: str
    elo_rating: int
    completed_modules: List[str]
    module_attempts: Dict[str, int]  # Track attempts per module
    last_activity: datetime

class ProgressionSystem:
    def __init__(self, course_data_path: str):
        # --- FIX: Load JSON data from the provided path ---
        try:
            with open(course_data_path, 'r') as f:
                loaded_data = json.load(f)
            # Assuming the structure is {"course": { ... }}
            self.course_data = loaded_data['course']
        except FileNotFoundError:
            print(f"Error: Course data file not found at {course_data_path}")
            raise
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {course_data_path}")
            raise
        except KeyError:
            print(f"Error: JSON structure incorrect. Expected top-level key 'course'.")
            raise

        self.categories = {cat['id']: cat for cat in self.course_data.get('categories', [])}
        self.modules = {mod['id']: mod for mod in self.course_data.get('modules', [])}
        self.metadata = self.course_data.get('metadata', {})

        # --- Validate required metadata structure ---
        if 'eloCalculation' not in self.metadata or \
           'bonusConditions' not in self.metadata['eloCalculation'] or \
           'penaltyConditions' not in self.metadata['eloCalculation']:
            raise ValueError("Error: Missing required 'eloCalculation' structure in metadata.")
        if len(self.metadata['eloCalculation']['bonusConditions']) < 3 or \
           len(self.metadata['eloCalculation']['penaltyConditions']) < 2:
             raise ValueError("Error: Insufficient 'bonusConditions' or 'penaltyConditions' in metadata.")


    def calculate_elo_gain(self,
                          module_id: str,
                          user: MockUser,
                          checklist_items_completed: int,
                          time_taken: timedelta,
                          is_first_attempt: bool) -> int:
        """Calculate ELO points gained from completing a module."""
        if module_id not in self.modules:
            print(f"Warning: Module ID '{module_id}' not found.")
            return 0
        module = self.modules[module_id]

        # --- Add checks for required keys ---
        if 'difficulty' not in module or \
           'baseCompletionPoints' not in module['difficulty'] or \
           'score' not in module['difficulty'] or \
           'bonusMultiplier' not in module['difficulty']:
            print(f"Warning: Missing difficulty information for module '{module_id}'.")
            return 0
        if 'rubric' not in module or 'checklistItems' not in module['rubric']:
             print(f"Warning: Missing rubric/checklist information for module '{module_id}'.")
             return 0

        difficulty = module['difficulty']
        checklist_items = module['rubric']['checklistItems']
        num_checklist_items = len(checklist_items)

        if num_checklist_items == 0:
             print(f"Warning: Module '{module_id}' has no checklist items.")
             return 0 # Avoid division by zero


        # Base calculation from metadata formula
        base_points = difficulty['baseCompletionPoints']
        difficulty_factor = difficulty['score'] / 5  # Assuming score is out of 5
        bonus_multiplier = difficulty['bonusMultiplier']
        checklist_factor = checklist_items_completed / num_checklist_items

        # Calculate base ELO gain
        elo_gain = base_points * difficulty_factor * bonus_multiplier * checklist_factor

        # Apply bonus conditions (ensure metadata structure exists)
        try:
            if checklist_items_completed == num_checklist_items:
                elo_gain *= self.metadata['eloCalculation']['bonusConditions'][0]['multiplier']

            # Assuming estimatedCompletionTime is stored in metadata, e.g., 2.5 hours
            # Replace with actual field if it exists elsewhere
            estimated_time_seconds = self.metadata.get('estimatedCompletionTimeSeconds', 2.5 * 3600)
            if time_taken <= timedelta(seconds=estimated_time_seconds):
                elo_gain *= self.metadata['eloCalculation']['bonusConditions'][1]['multiplier']

            if is_first_attempt:
                elo_gain *= self.metadata['eloCalculation']['bonusConditions'][2]['multiplier']

            # Apply penalty conditions
            if not is_first_attempt:
                elo_gain *= self.metadata['eloCalculation']['penaltyConditions'][0]['multiplier']

            # Missing crucial items (e.g., less than 70%)
            if checklist_items_completed < num_checklist_items * 0.7:
                elo_gain *= self.metadata['eloCalculation']['penaltyConditions'][1]['multiplier']
        except (IndexError, KeyError) as e:
             print(f"Warning: Missing configuration in metadata for ELO calculation bonuses/penalties: {e}")
             # Decide how to handle missing config: proceed without factor or return 0?
             # Here, we proceed without applying the missing factor.

        return int(elo_gain)

    def can_attempt_module(self, module_id: str, user: MockUser) -> bool:
        """Check if user meets requirements to attempt a module."""
        if module_id not in self.modules:
            return False
        module = self.modules[module_id]
        # --- Add check for required key ---
        if 'requiredEloRating' not in module:
            print(f"Warning: Missing 'requiredEloRating' for module '{module_id}'. Assuming 0.")
            return True # Or False, depending on desired default behavior
        return user.elo_rating >= module['requiredEloRating']

    def complete_module(self,
                       module_id: str,
                       user: MockUser,
                       checklist_items_completed: int,
                       time_taken: timedelta) -> Optional[int]:
        """Process module completion and return ELO points gained."""
        if module_id not in self.modules:
             print(f"Attempted to complete non-existent module: {module_id}")
             return None

        if not self.can_attempt_module(module_id, user):
            # Optionally print a message here if needed
            # print(f"User {user.name} cannot attempt module {module_id} yet (ELO: {user.elo_rating}, Required: {self.modules[module_id].get('requiredEloRating', 'N/A')})")
            return None # Return None to indicate failure to attempt

        is_first_attempt = user.module_attempts.get(module_id, 0) == 0
        user.module_attempts[module_id] = user.module_attempts.get(module_id, 0) + 1

        elo_gain = self.calculate_elo_gain(
            module_id=module_id,
            user=user,
            checklist_items_completed=checklist_items_completed,
            time_taken=time_taken,
            is_first_attempt=is_first_attempt
        )

        # Update user stats only if ELO gain is calculated (module exists, etc.)
        if elo_gain is not None: # calculate_elo_gain could return 0, which is valid
            user.elo_rating += elo_gain
            if module_id not in user.completed_modules:
                user.completed_modules.append(module_id)
            user.last_activity = datetime.now()

        return elo_gain

def generate_mock_users(num_users: int) -> List[MockUser]:
    """Generate a list of mock users with fixed initial ELO rating of 100."""
    users = []
    for i in range(num_users):
        users.append(MockUser(
            id=f"user_{i+1}",
            name=f"Test User {i+1}",
            elo_rating=100,  # Fixed initial ELO of 100
            completed_modules=[],
            module_attempts={},
            last_activity=datetime.now()
        ))
    return users

def create_mock_json(path="CoursesTYG.json"):
    """Creates a sample CoursesTYG.json file for testing."""
    mock_data = {
        "course": {
            "metadata": {
                "estimatedCompletionTimeSeconds": 9000, # 2.5 hours
                "eloCalculation": {
                    "bonusConditions": [
                        {"condition": "perfectScore", "multiplier": 1.2},
                        {"condition": "timeBonus", "multiplier": 1.1},
                        {"condition": "firstAttempt", "multiplier": 1.15}
                    ],
                    "penaltyConditions": [
                        {"condition": "repeatedAttempt", "multiplier": 0.9},
                        {"condition": "lowScore", "multiplier": 0.8}
                    ]
                }
            },
            "categories": [
                {"id": "cat1", "title": "Basics"},
                {"id": "cat2", "title": "Advanced"}
            ],
            "modules": [
                {
                    "id": "mod1",
                    "title": "Introduction",
                    "categoryId": "cat1",
                    "difficulty": {"score": 1, "baseCompletionPoints": 50, "bonusMultiplier": 1.0},
                    "requiredEloRating": 0,
                    "rubric": {"checklistItems": ["Task 1", "Task 2"]}
                },
                {
                    "id": "mod2",
                    "title": "Intermediate Concepts",
                    "categoryId": "cat1",
                    "difficulty": {"score": 3, "baseCompletionPoints": 100, "bonusMultiplier": 1.1},
                    "requiredEloRating": 120,
                    "rubric": {"checklistItems": ["Item A", "Item B", "Item C"]}
                },
                {
                    "id": "mod3",
                    "title": "Expert Techniques",
                    "categoryId": "cat2",
                    "difficulty": {"score": 5, "baseCompletionPoints": 150, "bonusMultiplier": 1.2},
                    "requiredEloRating": 250,
                    "rubric": {"checklistItems": ["Check 1", "Check 2", "Check 3", "Check 4"]}
                }
            ]
        }
    }
    try:
        with open(path, 'w') as f:
            json.dump(mock_data, f, indent=2)
        print(f"Created mock data file: {path}")
    except IOError as e:
        print(f"Error creating mock data file {path}: {e}")

def test_progression_system():
    # --- FIX: Define path relative to current working directory ---
    json_file_name = 'CoursesTYG.json'
    output_file_name = 'progression_results.txt'

    # --- Create a mock JSON file if it doesn't exist ---
    if not os.path.exists(json_file_name):
        create_mock_json(json_file_name)

    # --- Check if JSON file exists before proceeding ---
    if not os.path.exists(json_file_name):
         print(f"Error: {json_file_name} not found and could not be created. Aborting.")
         return # Exit if file is still missing

    # Initialize the progression system using the filename
    try:
        system = ProgressionSystem(json_file_name)
    except (FileNotFoundError, json.JSONDecodeError, KeyError, ValueError) as e:
        print(f"Failed to initialize ProgressionSystem: {e}")
        return # Exit if initialization fails


    # Generate mock users
    users = generate_mock_users(5)

    # Define output file path in the current working directory
    output_file_path = os.path.join(os.getcwd(), output_file_name)

    try:
        with open(output_file_path, 'w') as f:
            # Write header
            f.write("Testing Progression System:\n")
            f.write("-" * 50 + "\n")

            # --- Ensure modules are sorted or processed in a predictable order if needed ---
            # Sorting by required ELO might make sense for simulation
            module_ids_sorted = sorted(system.modules.keys(), key=lambda mid: system.modules[mid].get('requiredEloRating', 0))
            # If you prefer the order from the JSON file (dict order is insertion order in Python 3.7+):
            # module_ids_to_process = list(system.modules.keys())


            for user in users:
                f.write(f"\nTesting user: {user.name} (Initial ELO: {user.elo_rating})\n")

                # Try completing modules
                for module_id in module_ids_sorted: # Use the sorted list
                    module = system.modules[module_id]
                    if system.can_attempt_module(module_id, user):
                        # Simulate varying completion quality and time
                        checklist_items = module.get('rubric', {}).get('checklistItems', [])
                        num_checklist_items = len(checklist_items)

                        if num_checklist_items == 0:
                             f.write(f"  Skipped {module.get('title', module_id)} - No checklist items defined\n")
                             continue

                        # Simulate completing 70% to 100% of items
                        completed_items = random.randint(int(num_checklist_items * 0.7), num_checklist_items)
                        # Simulate time between 30 mins and 3 hours
                        time_taken = timedelta(minutes=random.randint(30, 180))

                        elo_gain = system.complete_module(
                            module_id=module_id,
                            user=user,
                            checklist_items_completed=completed_items,
                            time_taken=time_taken
                        )

                        # Check if complete_module returned a gain (i.e., attempt was allowed)
                        if elo_gain is not None:
                            f.write(f"  Completed {module.get('title', module_id)}:\n")
                            f.write(f"    - ELO gained: {elo_gain}\n")
                            f.write(f"    - New ELO: {user.elo_rating}\n")
                            f.write(f"    - Items completed: {completed_items}/{num_checklist_items}\n")
                            f.write(f"    - Time taken: {time_taken}\n")
                            f.write(f"    - Attempt #: {user.module_attempts[module_id]}\n")

                            # --- Optional: Simulate a second attempt with penalties ---
                            if random.random() < 0.2: # 20% chance of a second attempt
                                f.write(f"  -- Simulating second attempt for {module.get('title', module_id)} --\n")
                                completed_items_retry = random.randint(int(num_checklist_items * 0.8), num_checklist_items) # Maybe better score
                                time_taken_retry = timedelta(minutes=random.randint(30, 120)) # Maybe faster
                                elo_gain_retry = system.complete_module(
                                    module_id=module_id,
                                    user=user,
                                    checklist_items_completed=completed_items_retry,
                                    time_taken=time_taken_retry
                                )
                                if elo_gain_retry is not None:
                                     f.write(f"    - ELO gained (retry): {elo_gain_retry}\n")
                                     f.write(f"    - New ELO: {user.elo_rating}\n")
                                     f.write(f"    - Items completed (retry): {completed_items_retry}/{num_checklist_items}\n")
                                     f.write(f"    - Time taken (retry): {time_taken_retry}\n")
                                     f.write(f"    - Attempt #: {user.module_attempts[module_id]}\n")


                        # This 'else' should theoretically not be reached if can_attempt_module is True
                        # and complete_module returns None only if can_attempt_module was false.
                        # Kept for safety.
                        # else:
                        #    f.write(f"  Attempted {module.get('title', module_id)} but failed (e.g., ELO too low)\n")
                    else:
                        f.write(f"  Skipped {module.get('title', module_id)} - ELO requirement ({module.get('requiredEloRating', 'N/A')}) not met (User ELO: {user.elo_rating})\n")

            f.write("\nFinal Results:\n")
            f.write("-" * 50 + "\n")
            for user in users:
                f.write(f"{user.name}:\n")
                f.write(f"  Final ELO: {user.elo_rating}\n")
                f.write(f"  Modules completed: {len(user.completed_modules)} ({', '.join(user.completed_modules)})\n")
                f.write(f"  Total attempts: {sum(user.module_attempts.values())}\n")
                # f.write(f"  Attempts detail: {user.module_attempts}\n") # Uncomment for more detail

        # Also print to console for immediate feedback
        print(f"\nTest completed. Results saved to {output_file_path}")

        # Print a summary to console
        print("\nSummary of Results:")
        print("-" * 50)
        for user in users:
            print(f"{user.name}: Initial ELO 100, Final ELO {user.elo_rating}, Modules completed: {len(user.completed_modules)}")

    except IOError as e:
        print(f"Error writing results to {output_file_path}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during testing: {e}")
        import traceback
        traceback.print_exc() # Print detailed traceback for debugging


if __name__ == "__main__":
    # Make sure you have a 'CoursesTYG.json' file in the same directory
    # or run create_mock_json() first if needed.
    test_progression_system()

Created mock data file: CoursesTYG.json

Test completed. Results saved to /content/progression_results.txt

Summary of Results:
--------------------------------------------------
Test User 1: Initial ELO 100, Final ELO 113, Modules completed: 1
Test User 2: Initial ELO 100, Final ELO 115, Modules completed: 1
Test User 3: Initial ELO 100, Final ELO 105, Modules completed: 1
Test User 4: Initial ELO 100, Final ELO 415, Modules completed: 3
Test User 5: Initial ELO 100, Final ELO 115, Modules completed: 1


In [2]:

CoursesTYG =  {
  "course": {
    "id": "mobile-email-writing",
    "title": "Mobile Email Writing Course",
    "categories": [
      {
        "id": "academic",
        "name": "Academic Communications",
        "description": "Foundational emails related to academic activities and requests",
        "minEloRating": 100,
        "baseEloGain": 25
      },
      {
        "id": "professional",
        "name": "Professional Networking",
        "description": "Advanced emails for career development and networking",
        "minEloRating": 250,
        "baseEloGain": 40
      },
      {
        "id": "administrative",
        "name": "Administrative Requests",
        "description": "Intermediate emails for administrative changes and exceptions",
        "minEloRating": 175,
        "baseEloGain": 30
      }
    ],
    "modules": [
      {
        "id": "professor-recommendation",
        "title": "Professor Recommendation Request",
        "order": 1,
        "category": "academic",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Email subject line",
            "example": "Recommendation Request for Stanford Summer Research Program"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including greeting, main content, and signature",
            "example": "Dear Professor Chen,\n\nI hope this email finds you well. I am writing to request a letter of recommendation for the Stanford Summer Research Program. As a student in your Advanced Biology course last semester (Fall 2024) where I earned an A, I believe you could speak to my research capabilities and academic performance.\n\nBest regards,\nStudent Name"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Formal tone required",
            "Multiple attachments",
            "Timeline sensitivity"
          ],
          "baseCompletionPoints": 20,
          "bonusMultiplier": 1.2
        },
        "requiredEloRating": 100,
        "commonMistakes": [
          {
            "id": "m1-1",
            "description": "Sending the request too late (less than 2 weeks before deadline)"
          },
          {
            "id": "m1-2",
            "description": "Not providing specific details about the program or position being applied to"
          },
          {
            "id": "m1-3",
            "description": "Failing to mention specific class experiences or projects that demonstrate your capabilities"
          },
          {
            "id": "m1-4",
            "description": "Not offering alternative formats for the letter submission (online portal, email, physical mail)"
          }
        ],
        "scenario": {
          "context": "You need a recommendation letter for a summer research program at Stanford University. The deadline is in 3 weeks, and you took Professor Chen's Advanced Biology course last semester, earning an A.",
          "requirements": [
            "Your specific connection to the professor",
            "Program details",
            "Deadline",
            "Any materials you're attaching (resume, transcript)"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of how well you've established your academic connection and performance",
            "Evaluation of clarity in program details and timeline presentation",
            "Assessment of professionalism and courtesy in the request"
          ],
          "checklistItems": [
            {
              "id": "c1-1",
              "description": "Mentioned specific course name, semester, and grade"
            },
            {
              "id": "c1-2",
              "description": "Included clear program deadline and submission method"
            },
            {
              "id": "c1-3",
              "description": "Attached all referenced documents (resume, transcript)"
            },
            {
              "id": "c1-4",
              "description": "Provided program name and specific position details"
            },
            {
              "id": "c1-5",
              "description": "Included a proposed follow-up timeline"
            }
          ]
        },
        "contentCards": [
          "Start your request at least 3-4 weeks before the deadline",
          "Research the program thoroughly to provide specific details",
          "Prepare your resume and transcript before making the request",
          "Consider the professor's schedule and workload",
          "Be specific about why you chose this professor",
          "Use a professional email address",
          "Include your full name and student ID",
          "Mention specific achievements or projects",
          "Offer to meet in person to discuss the request",
          "Follow up politely if no response within 3-4 days",
          "Remember that professors receive many recommendation requests. Make yours stand out by being specific, professional, and showing you've done your homework."
        ],
        "resourceCards": [
          "Q: How long should I wait for a response?\nA: Wait 3-4 business days before following up. If urgent, you can send a gentle reminder after 2 days.",
          "Q: Should I attach my resume in the first email?\nA: Yes, it's helpful to attach your resume and any relevant documents in the initial request to make it easier for the professor to write a detailed recommendation.",
          "Q: What if the professor declines?\nA: Thank them for their time and ask if they can suggest alternative recommenders. Always maintain professionalism and gratitude."
        ]
      },
      {
        "id": "advisor-recommendation",
        "title": "Academic Advisor Recommendation",
        "order": 2,
        "category": "academic",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Email subject line",
            "example": "Thank You and NASA JPL Recommendation Request"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including gratitude and recommendation request",
            "example": "Dear Dr. Martinez,\n\nI wanted to express my sincere gratitude for your mentorship over the past two years. Your guidance has been instrumental in my academic journey.\n\nI am applying for the NASA JPL internship and would greatly appreciate a letter of recommendation that highlights our research work together.\n\nBest regards,\nStudent Name"
          }
        },
        "difficulty": {
          "score": 3,
          "scale": 5,
          "factors": [
            "Personal relationship balance",
            "Long-term relationship context",
            "Complex achievement documentation"
          ],
          "baseCompletionPoints": 25,
          "bonusMultiplier": 1.3
        },
        "requiredEloRating": 120,
        "commonMistakes": [
          {
            "id": "m2-1",
            "description": "Forgetting to express gratitude for previous guidance before making the request"
          },
          {
            "id": "m2-2",
            "description": "Not explaining why you're specifically asking this advisor (vs others)"
          },
          {
            "id": "m2-3",
            "description": "Not providing a clear timeline for when you need the recommendation"
          },
          {
            "id": "m2-4",
            "description": "Failing to mention specific achievements or growth under their advisorship"
          }
        ],
        "scenario": {
          "context": "You've worked with your advisor Dr. Martinez for two years. You're applying for a competitive internship at NASA's Jet Propulsion Laboratory and need a recommendation highlighting your research experience.",
          "requirements": [
            "Thanks them for their mentorship",
            "Requests a recommendation with specific details about the internship"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of how effectively you've highlighted your research relationship",
            "Evaluation of gratitude expression and personal growth demonstration",
            "Assessment of clarity in internship relevance to your path"
          ],
          "checklistItems": [
            {
              "id": "c2-1",
              "description": "Expressed specific examples of valuable mentorship"
            },
            {
              "id": "c2-2",
              "description": "Included detailed internship program information"
            },
            {
              "id": "c2-3",
              "description": "Mentioned specific research projects or achievements"
            },
            {
              "id": "c2-4",
              "description": "Provided clear deadline and submission instructions"
            },
            {
              "id": "c2-5",
              "description": "Included a graceful 'out' if they're unable to help"
            }
          ]
        },
        "contentCards": [
          "Review your past interactions and achievements with the advisor",
          "Prepare a list of specific projects or milestones",
          "Have your internship/job description ready",
          "Consider the timing of your request",
          "Be ready to provide context about your goals",
          "Express genuine gratitude for past guidance",
          "Be specific about the opportunity you're pursuing",
          "Provide clear deadlines and requirements",
          "Offer to meet in person to discuss details",
          "Include relevant documents (resume, transcript, project descriptions)",
          "Advisors often write many recommendations, so make your request memorable by highlighting your unique journey and achievements under their guidance."
        ],
        "resourceCards": [
          "Q: How far in advance should I request the recommendation?\nA: Request at least 3 weeks before the deadline, but earlier is better as advisors often have many requests during peak application seasons.",
          "Q: Should I provide a draft of the recommendation?\nA: While not required, providing bullet points of key achievements and experiences can help the advisor write a more detailed and accurate recommendation.",
          "Q: What if I haven't interacted with my advisor recently?\nA: Schedule a meeting to catch up and discuss your goals before making the request. This helps refresh their memory of your work and commitment."
        ]
      },
      {
        "id": "illness-absence",
        "title": "Illness-Related Absence",
        "order": 3,
        "category": "administrative",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Email subject line",
            "example": "Absence Notice - Marketing Strategy Presentation (Feb 20)"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including situation explanation and makeup plan",
            "example": "Dear Professor Thompson,\n\nI regret to inform you that I have developed a high fever and will not be able to attend tomorrow's Marketing Strategy presentation. I have already informed my group members and can present my portion next week.\n\nBest regards,\nStudent Name"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Time sensitivity",
            "Group impact consideration",
            "Professional discretion"
          ],
          "baseCompletionPoints": 15,
          "bonusMultiplier": 1.1
        },
        "requiredEloRating": 125,
        "commonMistakes": [
          {
            "id": "m3-1",
            "description": "Being too casual or providing unnecessary medical details"
          },
          {
            "id": "m3-2",
            "description": "Not proposing a plan to catch up on missed work"
          },
          {
            "id": "m3-3",
            "description": "Waiting too long to notify the professor about the absence"
          },
          {
            "id": "m3-4",
            "description": "Not acknowledging the impact on team members or class activities"
          }
        ],
        "scenario": {
          "context": "You've developed a high fever and won't be able to attend tomorrow's important presentation in your Marketing Strategy class.",
          "requirements": [
            "Professionally explains your situation",
            "Proposes a make-up date",
            "Addresses how you'll handle any group work impacts"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of professionalism in explaining the situation",
            "Evaluation of proactiveness in providing solutions",
            "Assessment of consideration for group impact"
          ],
          "checklistItems": [
            {
              "id": "c3-1",
              "description": "Stated absence timing and duration clearly"
            },
            {
              "id": "c3-2",
              "description": "Proposed specific make-up presentation date(s)"
            },
            {
              "id": "c3-3",
              "description": "Outlined plan for group work coordination"
            },
            {
              "id": "c3-4",
              "description": "Mentioned willingness to provide documentation if needed"
            },
            {
              "id": "c3-5",
              "description": "Included plan to catch up on missed material"
            }
          ]
        },
        "contentCards": [
          "Notify as early as possible",
          "Be concise but informative",
          "Have a plan ready for catching up",
          "Consider group dynamics",
          "Keep medical details private",
          "Use a clear subject line with date and class",
          "Include specific missed activities",
          "Propose concrete make-up plans",
          "Offer to provide documentation if needed",
          "Follow up with gratitude after resolution",
          "While illness is unavoidable, being proactive and professional in your communication can help maintain good standing with your professors and classmates."
        ],
        "resourceCards": [
          "Q: Do I need to provide medical documentation?\nA: Only if specifically requested by the professor or university policy. Generally, a professional explanation is sufficient for short absences.",
          "Q: How should I handle group work absences?\nA: Notify your group members immediately, provide any work you've completed, and propose specific ways to contribute remotely or make up the work.",
          "Q: What if I miss a major assignment deadline?\nA: Contact the professor as soon as possible, explain the situation professionally, and propose a specific timeline for submission."
        ]
      },
      {
        "id": "introduction-request",
        "title": "Professional Introduction Request",
        "order": 4,
        "category": "professional",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Email subject line",
            "example": "Introduction Request - Apple Design Team Connection"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including research, value proposition, and forwardable introduction",
            "example": "Dear [Mentor Name],\n\nI noticed your LinkedIn connection with Jane Smith, Head of Design at Apple. I've been following her work on user-centered design principles and would greatly appreciate an introduction to learn more about her career path.\n\nBest regards,\n[Your name]"
          }
        },
        "difficulty": {
          "score": 4,
          "scale": 5,
          "factors": [
            "Complex networking dynamics",
            "Research requirements",
            "Double audience consideration"
          ],
          "baseCompletionPoints": 30,
          "bonusMultiplier": 1.4
        },
        "requiredEloRating": 155,
        "commonMistakes": [
          {
            "id": "m4-1",
            "description": "Not explaining how you found the person you want to be introduced to"
          },
          {
            "id": "m4-2",
            "description": "Making the request too demanding on the connector's time"
          },
          {
            "id": "m4-3",
            "description": "Failing to provide a draft introduction message that can be forwarded"
          },
          {
            "id": "m4-4",
            "description": "Not demonstrating value or mutual benefit in the connection"
          }
        ],
        "scenario": {
          "context": "You noticed your alumni mentor is connected to the Head of Design at Apple, and you'd love to learn more about their career path.",
          "requirements": [
            "Explains your specific interest",
            "Makes it easy for your mentor to forward/connect",
            "Shows you've done your research"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of how well you've demonstrated your research",
            "Evaluation of the value proposition for both parties",
            "Assessment of how easy you've made it for the connector"
          ],
          "checklistItems": [
            {
              "id": "c4-1",
              "description": "Explained specific reason for wanting to connect"
            },
            {
              "id": "c4-2",
              "description": "Included a draft forward-able message"
            },
            {
              "id": "c4-3",
              "description": "Demonstrated knowledge of target contact's work"
            },
            {
              "id": "c4-4",
              "description": "Provided clear, specific ask for the introduction"
            },
            {
              "id": "c4-5",
              "description": "Included your relevant background briefly"
            }
          ]
        },
        "contentCards": [
          "Research both the target contact and your connector",
          "Prepare a clear value proposition",
          "Have a specific ask ready",
          "Consider timing and context",
          "Be ready to follow up professionally",
          "Make it easy for your connector to forward your request",
          "Show genuine interest in the target contact's work",
          "Be specific about what you want to learn",
          "Include relevant background information",
          "Offer to meet at their convenience",
          "The key to successful networking is making it easy and beneficial for all parties involved. Focus on creating value in the connection."
        ],
        "resourceCards": [
          "Q: How long should my introduction request be?\nA: Keep it concise (2-3 paragraphs) but include all essential information: who you are, why you want to connect, and what you hope to learn.",
          "Q: Should I follow up if I don't hear back?\nA: Wait 5-7 business days before sending a polite follow-up to your connector. Don't contact the target person directly unless specifically invited.",
          "Q: What if my connector doesn't respond?\nA: Wait 3-4 days before sending a gentle reminder. If still no response, thank them for considering and explore alternative networking opportunities."
        ]
      },
      {
        "id": "assignment-resubmission",
        "title": "Assignment Resubmission Request",
        "order": 5,
        "category": "administrative",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Email subject line",
            "example": "Database Management Final Project - Technical Issue and Resubmission Request"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including issue explanation, documentation, and resubmission proposal",
            "example": "Dear Professor Johnson,\n\nI experienced a technical issue while submitting my Database Management final project. My laptop crashed during submission at 11:58 PM, causing me to miss the midnight deadline. I have attached the IT support documentation and would like to request permission to resubmit.\n\nBest regards,\n[Your name]"
          }
        },
        "difficulty": {
          "score": 3,
          "scale": 5,
          "factors": [
            "Technical documentation",
            "Policy navigation",
            "Urgency management"
          ],
          "baseCompletionPoints": 20,
          "bonusMultiplier": 1.2
        },
        "requiredEloRating": 145,
        "commonMistakes": [
          {
            "id": "m5-1",
            "description": "Not taking full responsibility for the situation"
          },
          {
            "id": "m5-2",
            "description": "Making excuses instead of providing solutions"
          },
          {
            "id": "m5-3",
            "description": "Failing to provide proper documentation or evidence of the technical issue"
          },
          {
            "id": "m5-4",
            "description": "Not acknowledging the instructor's time and effort in accommodating the request"
          }
        ],
        "scenario": {
          "context": "Your laptop crashed while submitting your final project for Database Management, causing you to miss the deadline by 2 hours. You have documentation from IT support about the incident.",
          "requirements": [
            "Explains the situation concisely",
            "Provides documentation",
            "Proposes a specific resubmission timeline",
            "Acknowledges any late penalties that may apply"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of responsibility acknowledgment",
            "Evaluation of solution-focused approach",
            "Assessment of documentation and timeline clarity"
          ],
          "checklistItems": [
            {
              "id": "c5-1",
              "description": "Clearly explained the technical issue timing"
            },
            {
              "id": "c5-2",
              "description": "Attached relevant documentation/evidence"
            },
            {
              "id": "c5-3",
              "description": "Proposed specific resubmission timeline"
            },
            {
              "id": "c5-4",
              "description": "Acknowledged any applicable penalties"
            },
            {
              "id": "c5-5",
              "description": "Included preventive measures for future submissions"
            }
          ]
        },
        "contentCards": [
          "Document the issue immediately",
          "Gather all relevant evidence",
          "Review course policies",
          "Prepare a clear timeline",
          "Have a backup plan ready",
          "Take full responsibility",
          "Be solution-focused",
          "Provide clear documentation",
          "Propose specific make-up plans",
          "Learn from the experience",
          "While technical issues are frustrating, handling them professionally and proactively can help maintain good standing with your professors."
        ],
        "resourceCards": [
          "Q: What documentation should I provide?\nA: Include screenshots, error messages, IT support tickets, or any other evidence that clearly shows the technical issue and its timing.",
          "Q: Will I receive a grade penalty?\nA: This depends on your professor's policies. Be prepared to accept any applicable penalties while demonstrating your commitment to completing the work.",
          "Q: How can I prevent this in the future?\nA: Submit assignments well before deadlines, use reliable devices/networks, and keep backup copies of your work."
        ]
      },
      {
        "id": "scholarship-followup",
        "title": "Scholarship Application Follow-up",
        "order": 6,
        "category": "academic",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for your follow-up email",
            "example": "Scholarship Application Status Inquiry - Fall 2025"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content requesting status update and expressing continued interest",
            "example": "Dear Scholarship Committee,\n\nI hope you are well. I submitted my application for the Fall 2025 Academic Excellence Scholarship two weeks ago and would like to kindly inquire about its status. I remain very interested in this opportunity and appreciate any update you can provide.\n\nThank you for your time and consideration.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Formal tone maintenance",
            "Precision of inquiry",
            "Timeliness of follow-up"
          ],
          "baseCompletionPoints": 15,
          "bonusMultiplier": 1.1
        },
        "requiredEloRating": 130,
        "commonMistakes": [
          {
            "id": "m6-1",
            "description": "Being too pushy or demanding about the decision"
          },
          {
            "id": "m6-2",
            "description": "Forgetting to restate key application details"
          },
          {
            "id": "m6-3",
            "description": "Neglecting to show appreciation for the committee’s time"
          },
          {
            "id": "m6-4",
            "description": "Not including any follow-up action or contact info"
          }
        ],
        "scenario": {
          "context": "You applied for a scholarship 2 weeks ago and haven't heard back. You want to politely follow up without appearing impatient.",
          "requirements": [
            "Reference the scholarship and date of submission",
            "Politely request any status update",
            "Express continued enthusiasm"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of courteous language and polite tone",
            "Evaluation of clarity in referencing the original application",
            "Assessment of how well you express continued interest"
          ],
          "checklistItems": [
            {
              "id": "c6-1",
              "description": "Mentioned specific scholarship name and submission date"
            },
            {
              "id": "c6-2",
              "description": "Maintained respectful and professional tone"
            },
            {
              "id": "c6-3",
              "description": "Clearly requested a status update"
            },
            {
              "id": "c6-4",
              "description": "Expressed gratitude for their time"
            },
            {
              "id": "c6-5",
              "description": "Included your contact information or reference number"
            }
          ]
        },
        "contentCards": [
          "Send follow-up emails at least 1-2 weeks after submission",
          "Keep your email short and polite",
          "Include relevant application details",
          "Maintain a respectful tone, avoiding pressure",
          "Consider any stated response timelines",
          "Thank the committee for their time and consideration",
          "Re-emphasize your interest in the scholarship",
          "Provide any additional information if requested",
          "Double-check for grammar and clarity",
          "Use a formal salutation and closing",
          "Remember, scholarship committees often receive numerous applications, so patience and professionalism are key."
        ],
        "resourceCards": [
          "Q: Is it too soon to follow up after 2 weeks?\nA: Generally, 2 weeks is a reasonable timeframe. However, always check if the scholarship guidelines specify a different timeline for updates.",
          "Q: What if I still don't hear back?\nA: Wait another week before sending a second follow-up. If there’s still no response, you may contact the institution’s admissions or financial aid office.",
          "Q: Should I attach my application documents again?\nA: Not unless specifically requested. A reference to your previous submission should suffice."
        ]
      },
      {
        "id": "study-group-invitation",
        "title": "Study Group Invitation",
        "order": 7,
        "category": "academic",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for the study group invitation",
            "example": "Invitation: ECON 201 Study Group Meeting"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content inviting classmates to a study group session",
            "example": "Hello ECON 201 classmates,\n\nI hope everyone's semester is going well. I'm organizing a study group to review chapters 3-5 for our upcoming midterm. We plan to meet on Wednesday at 4 PM in the library study room. If you'd like to join or have suggestions, please let me know.\n\nBest,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 1,
          "scale": 5,
          "factors": [
            "Informal yet organized tone",
            "Coordinating schedules",
            "Clear call-to-action"
          ],
          "baseCompletionPoints": 10,
          "bonusMultiplier": 1.0
        },
        "requiredEloRating": 110,
        "commonMistakes": [
          {
            "id": "m7-1",
            "description": "Forgetting to specify date, time, or location"
          },
          {
            "id": "m7-2",
            "description": "Using a confusing or overly casual tone"
          },
          {
            "id": "m7-3",
            "description": "Not clarifying the topics or purpose of the study group"
          },
          {
            "id": "m7-4",
            "description": "Failing to provide a way for classmates to RSVP or ask questions"
          }
        ],
        "scenario": {
          "context": "You want to form a study group with classmates to prepare for an upcoming exam in ECON 201. You're sending an initial invitation email.",
          "requirements": [
            "Mention the course and exam topics",
            "Provide date, time, and location",
            "Offer a way to confirm attendance or suggest changes"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of clarity in date/time/location details",
            "Evaluation of inclusivity and invitation tone",
            "Assessment of whether a clear call-to-action (RSVP) is provided"
          ],
          "checklistItems": [
            {
              "id": "c7-1",
              "description": "Specified course name and chapters/topics"
            },
            {
              "id": "c7-2",
              "description": "Included exact meeting date, time, and location"
            },
            {
              "id": "c7-3",
              "description": "Provided a method for classmates to respond"
            },
            {
              "id": "c7-4",
              "description": "Used an inviting yet organized tone"
            },
            {
              "id": "c7-5",
              "description": "Encouraged collaboration or topic suggestions"
            }
          ]
        },
        "contentCards": [
          "Mention relevant course material",
          "Keep the invitation concise",
          "Be flexible with scheduling",
          "Encourage participation by inviting suggestions",
          "Use a friendly tone but remain organized",
          "Follow up with a reminder if needed",
          "Consider offering snacks or refreshments if appropriate",
          "Respect everyone's time by starting and ending on schedule",
          "If many are interested, consider rotating meeting locations",
          "Building a collaborative environment can benefit everyone’s understanding and performance"
        ],
        "resourceCards": [
          "Q: Should I include the professor in the invitation?\nA: Generally, study groups are student-led, but you can CC the professor if you feel it might be beneficial or if they've encouraged group study.",
          "Q: How often should a study group meet?\nA: This depends on everyone’s schedules and exam timelines. Meeting once a week or bi-weekly is common before major exams.",
          "Q: What if only a few people respond?\nA: A smaller study group can still be effective. Focus on consistent scheduling and shared resources to keep everyone engaged."
        ]
      },
      {
        "id": "internship-application",
        "title": "Internship Application Email",
        "order": 8,
        "category": "professional",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for internship application",
            "example": "Summer UX Internship Application - [Your Name]"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content including your background, motivation, and attached resume",
            "example": "Dear Hiring Team,\n\nI hope you are doing well. My name is [Your Name], a junior majoring in Computer Science with a strong interest in UX design. I am reaching out to apply for the Summer UX Internship at XYZ Company. I have attached my resume and a brief portfolio of design projects.\n\nThank you for your consideration, and I look forward to the possibility of working with your team.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 3,
          "scale": 5,
          "factors": [
            "Professional branding",
            "Highlighting relevant skills",
            "Concise yet impactful"
          ],
          "baseCompletionPoints": 25,
          "bonusMultiplier": 1.3
        },
        "requiredEloRating": 160,
        "commonMistakes": [
          {
            "id": "m8-1",
            "description": "Failing to properly address the hiring manager or team"
          },
          {
            "id": "m8-2",
            "description": "Not specifying the position and session (e.g., summer internship)"
          },
          {
            "id": "m8-3",
            "description": "Using an unprofessional or personal tone"
          },
          {
            "id": "m8-4",
            "description": "Forgetting to attach a resume or portfolio"
          }
        ],
        "scenario": {
          "context": "You are a junior looking to apply for a summer internship in UX design. You need to showcase your relevant skills and attach your resume or portfolio.",
          "requirements": [
            "State your academic background briefly",
            "Mention relevant UX/design skills",
            "Attach your resume/portfolio"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of how well you present relevant skills",
            "Evaluation of professional tone and clarity",
            "Assessment of readiness of attachments and contact details"
          ],
          "checklistItems": [
            {
              "id": "c8-1",
              "description": "Properly addressed the hiring team or manager"
            },
            {
              "id": "c8-2",
              "description": "Mentioned exact internship role and timing"
            },
            {
              "id": "c8-3",
              "description": "Highlighted skills or coursework related to UX design"
            },
            {
              "id": "c8-4",
              "description": "Attached resume and/or portfolio link"
            },
            {
              "id": "c8-5",
              "description": "Included professional closing and signature with contact information"
            }
          ]
        },
        "contentCards": [
          "Use a clear subject line specifying the internship title",
          "Research the company to tailor your email",
          "Highlight relevant coursework and projects",
          "Keep the body concise but specific",
          "Show genuine enthusiasm for the role",
          "Double-check attachments for accuracy",
          "Use a formal greeting (e.g., 'Dear Hiring Team')",
          "End with a polite closing (e.g., 'Sincerely')",
          "Proofread thoroughly for grammar and spelling",
          "Follow up after 1-2 weeks if no response",
          "A well-crafted internship email can set the stage for a successful professional relationship."
        ],
        "resourceCards": [
          "Q: How should I address the hiring manager if I don't know their name?\nA: Use a general but polite greeting such as 'Dear Hiring Team' or 'Dear [Department] Hiring Manager.'",
          "Q: How long should the email be?\nA: Aim for 2-3 short paragraphs, ensuring all key points are covered while maintaining clarity and brevity.",
          "Q: Should I mention specific projects?\nA: Yes, briefly highlight one or two relevant projects that demonstrate your skills and passion. Link to your portfolio for more details."
        ]
      },
      {
        "id": "event-networking",
        "title": "Post-Event Networking Follow-Up",
        "order": 9,
        "category": "professional",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for a follow-up after a networking event",
            "example": "Great Connecting at the Tech Panel Last Night"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content thanking a contact for the conversation and expressing interest in further dialogue",
            "example": "Hello [Name],\n\nIt was great meeting you at the Tech Panel last night. I appreciated your insights on blockchain development. I'd love to continue our conversation, especially regarding your work with decentralized finance platforms.\n\nHope to stay in touch,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Professional yet personable tone",
            "Recalling specific conversation points",
            "Prompt follow-up"
          ],
          "baseCompletionPoints": 15,
          "bonusMultiplier": 1.1
        },
        "requiredEloRating": 180,
        "commonMistakes": [
          {
            "id": "m9-1",
            "description": "Sending a generic follow-up without referencing the specific event or discussion"
          },
          {
            "id": "m9-2",
            "description": "Waiting too long to follow up (over a week)"
          },
          {
            "id": "m9-3",
            "description": "Being overly formal or too casual"
          },
          {
            "id": "m9-4",
            "description": "Failing to include a clear next step or request"
          }
        ],
        "scenario": {
          "context": "You met a professional contact at a tech panel who shared valuable insights. You want to maintain the connection and possibly discuss collaboration or advice.",
          "requirements": [
            "Thank them for their time",
            "Reference something specific from your conversation",
            "Propose or invite next steps (call, meeting, info exchange)"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of timeliness and personalization",
            "Evaluation of clarity in next-step invitation",
            "Assessment of friendly yet professional tone"
          ],
          "checklistItems": [
            {
              "id": "c9-1",
              "description": "Referenced the event and conversation topic"
            },
            {
              "id": "c9-2",
              "description": "Thanked them for their insights or time"
            },
            {
              "id": "c9-3",
              "description": "Suggested a follow-up action or meeting"
            },
            {
              "id": "c9-4",
              "description": "Kept the tone professional but friendly"
            },
            {
              "id": "c9-5",
              "description": "Included your contact information"
            }
          ]
        },
        "contentCards": [
          "Send your follow-up within 24-48 hours",
          "Be genuine and specific about what you learned",
          "Suggest a brief call or meeting if appropriate",
          "Keep the email concise and polite",
          "Offer value where possible",
          "Mention any relevant shared interests",
          "Avoid generic phrases like 'nice to meet you' without details",
          "Close with a polite invitation to continue the conversation",
          "Proofread before sending",
          "Personalization is key in networking; show that you remember them as an individual."
        ],
        "resourceCards": [
          "Q: Should I mention potential collaboration right away?\nA: If it feels appropriate, yes. But keep it light and exploratory rather than pushing them into a commitment.",
          "Q: What if I only have their LinkedIn, not their email?\nA: Send a personalized message on LinkedIn referencing your conversation. The same principles apply to any networking follow-up.",
          "Q: How long should I wait before sending a reminder if there's no response?\nA: Give it about a week before politely following up, referencing your initial message."
        ]
      },
      {
        "id": "grade-dispute",
        "title": "Grade Dispute Request",
        "order": 10,
        "category": "administrative",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for requesting a grade review",
            "example": "Grade Review Request - History 304 Midterm"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Complete email content politely disputing or requesting a review of a grade",
            "example": "Dear Professor Smith,\n\nI hope you're doing well. I received my midterm grade for History 304 and noticed an unexpected discrepancy in the essay portion. I have attached my exam copy and would appreciate an opportunity to discuss or clarify any concerns.\n\nThank you for your time and understanding.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 3,
          "scale": 5,
          "factors": [
            "Tactful approach",
            "Evidence-based argument",
            "Policy awareness"
          ],
          "baseCompletionPoints": 20,
          "bonusMultiplier": 1.2
        },
        "requiredEloRating": 150,
        "commonMistakes": [
          {
            "id": "m10-1",
            "description": "Using a confrontational or accusatory tone"
          },
          {
            "id": "m10-2",
            "description": "Not referring to specific parts of the exam or rubric"
          },
          {
            "id": "m10-3",
            "description": "Failing to attach evidence or your exam copy"
          },
          {
            "id": "m10-4",
            "description": "Ignoring official grade dispute policies or deadlines"
          }
        ],
        "scenario": {
          "context": "You believe there may have been a grading error on your History 304 midterm essay. You want to clarify or dispute the score in a polite, evidence-based manner.",
          "requirements": [
            "Mention the specific assessment",
            "Attach relevant documents",
            "Request a respectful review of the grade"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of politeness and respect",
            "Evaluation of how clearly you present evidence",
            "Assessment of awareness of policies and willingness to follow them"
          ],
          "checklistItems": [
            {
              "id": "c10-1",
              "description": "Specifically identified the assignment/exam in question"
            },
            {
              "id": "c10-2",
              "description": "Attached or referenced relevant exam sections"
            },
            {
              "id": "c10-3",
              "description": "Maintained a polite, non-confrontational tone"
            },
            {
              "id": "c10-4",
              "description": "Acknowledged possible misunderstandings or need for clarification"
            },
            {
              "id": "c10-5",
              "description": "Followed any known dispute policy or timeline"
            }
          ]
        },
        "contentCards": [
          "Review the grading policy before contacting your professor",
          "Gather specific evidence or examples of discrepancies",
          "Keep your tone respectful and open-minded",
          "Propose a meeting or office hours visit if needed",
          "Attach or reference the graded work",
          "Acknowledge potential misunderstandings politely",
          "Thank the professor for their time in reviewing",
          "Be prepared for either outcome",
          "Follow up if you don’t hear back within a reasonable timeframe",
          "A fact-based, polite approach often yields better outcomes in grade disputes."
        ],
        "resourceCards": [
          "Q: What if the professor refuses to review my grade?\nA: Check your institution’s policies. There may be a formal appeals process or an academic advisor you can consult.",
          "Q: Should I CC the department head or dean?\nA: Generally, start by emailing the professor directly. Escalate only if policy suggests doing so, or if no resolution is reached.",
          "Q: How soon after receiving my grade should I send a dispute request?\nA: As soon as possible. Most institutions have a specific window (e.g., within 1-2 weeks of receiving the grade)."
        ]
      },
      {
        "id": "course-overload-request",
        "title": "Course Overload Permission",
        "order": 11,
        "category": "administrative",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for requesting a course overload",
            "example": "Request for Course Overload Approval - Spring 2026"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content explaining reasons for enrolling in extra credits",
            "example": "Dear Academic Affairs Committee,\n\nI hope this message finds you well. I am currently a junior majoring in Business Administration and would like to enroll in 21 credits this Spring 2026 semester. My goal is to complete an additional research elective that aligns with my career aspirations in data analytics.\n\nI have maintained a 3.8 GPA and have the support of my academic advisor, Dr. Blake, whose letter is attached. I appreciate your consideration of this request.\n\nThank you,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 4,
          "scale": 5,
          "factors": [
            "Academic policy navigation",
            "Demonstrating capability",
            "Gathering advisor support"
          ],
          "baseCompletionPoints": 30,
          "bonusMultiplier": 1.4
        },
        "requiredEloRating": 165,
        "commonMistakes": [
          {
            "id": "m11-1",
            "description": "Not showing evidence of academic preparedness (e.g., GPA or advisor support)"
          },
          {
            "id": "m11-2",
            "description": "Failing to specify the reason or benefit of the extra credits"
          },
          {
            "id": "m11-3",
            "description": "Ignoring the official overload policy or not mentioning prerequisites"
          },
          {
            "id": "m11-4",
            "description": "Being too informal or demanding in tone"
          }
        ],
        "scenario": {
          "context": "You want to enroll in 21 credits next semester to complete a special research elective. You have a strong GPA and advisor support, and now need official permission.",
          "requirements": [
            "Provide your academic standing (e.g., GPA, year)",
            "Explain why you need extra credits",
            "Mention advisor endorsement if available"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of clarity in academic reasoning",
            "Evaluation of tone and adherence to policy",
            "Assessment of supporting evidence (e.g., advisor letter, GPA)"
          ],
          "checklistItems": [
            {
              "id": "c11-1",
              "description": "Stated current academic status and GPA"
            },
            {
              "id": "c11-2",
              "description": "Explained clearly why extra credits are needed"
            },
            {
              "id": "c11-3",
              "description": "Attached or mentioned advisor support"
            },
            {
              "id": "c11-4",
              "description": "Referred to relevant policy or procedure"
            },
            {
              "id": "c11-5",
              "description": "Maintained a professional and respectful tone"
            }
          ]
        },
        "contentCards": [
          "Check your institution’s maximum credit policies",
          "Compile a compelling reason (graduation timeline, special program)",
          "Highlight strong academic performance or relevant experience",
          "Obtain a support letter from your advisor if possible",
          "Use clear and concise language",
          "Refer to specific policy codes if known",
          "Be prepared to adjust if the committee imposes conditions",
          "Express gratitude for their time and review",
          "Allow enough lead time for processing the request",
          "Demonstrating preparedness and responsibility is key to securing an overload approval."
        ],
        "resourceCards": [
          "Q: Can I request an overload if my GPA is below 3.0?\nA: Policies vary, but most institutions require a solid GPA. Consult your advisor or department guidelines.",
          "Q: What if the committee denies my request?\nA: Inquire politely about their reasons and see if an appeal or alternative arrangement is possible.",
          "Q: How quickly should I expect a response?\nA: This depends on your institution’s procedures. Generally, allow 1-2 weeks, and follow up if necessary."
        ]
      },
      {
        "id": "apology-late-submission",
        "title": "Apology for Late Assignment Submission",
        "order": 12,
        "category": "administrative",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for an apology email regarding late submission",
            "example": "Apology and Late Submission - English Literature Essay"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content apologizing and providing a brief explanation for late work",
            "example": "Dear Professor Green,\n\nI would like to sincerely apologize for submitting my English Literature essay past the deadline. I experienced unexpected family circumstances that prevented me from completing the essay on time. I understand any penalties that may apply.\n\nThank you for your understanding.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Sincerity of apology",
            "Taking responsibility",
            "Respecting policy on late work"
          ],
          "baseCompletionPoints": 15,
          "bonusMultiplier": 1.1
        },
        "requiredEloRating": 135,
        "commonMistakes": [
          {
            "id": "m12-1",
            "description": "Offering excessive excuses instead of a concise explanation"
          },
          {
            "id": "m12-2",
            "description": "Failing to acknowledge the late penalty policy"
          },
          {
            "id": "m12-3",
            "description": "Using an overly casual or insincere tone"
          },
          {
            "id": "m12-4",
            "description": "Not proposing a plan to prevent future lateness"
          }
        ],
        "scenario": {
          "context": "You submitted an important assignment one day late due to unforeseen circumstances. You need to apologize and acknowledge any academic penalty.",
          "requirements": [
            "Acknowledge the late submission",
            "Offer a genuine apology",
            "Accept policy-based penalties"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of sincerity in apology",
            "Evaluation of personal responsibility taken",
            "Assessment of acknowledgement of potential penalties"
          ],
          "checklistItems": [
            {
              "id": "c12-1",
              "description": "Clearly stated the assignment and date of late submission"
            },
            {
              "id": "c12-2",
              "description": "Apologized sincerely without excessive detail"
            },
            {
              "id": "c12-3",
              "description": "Acknowledged the professor’s or institution’s late policy"
            },
            {
              "id": "c12-4",
              "description": "Expressed willingness to accept the penalty"
            },
            {
              "id": "c12-5",
              "description": "Mentioned preventive measures for future"
            }
          ]
        },
        "contentCards": [
          "Send your apology as soon as possible after missing the deadline",
          "Keep your explanation brief and honest",
          "Acknowledge the professor’s time and effort",
          "Offer any supporting documents only if necessary",
          "Accept the consequences gracefully",
          "Reassure the professor you will prevent future recurrences",
          "Do not blame others or external factors excessively",
          "If you need an extension, politely request it in advance next time",
          "Follow up if needed, but don’t badger the professor",
          "Taking responsibility promptly can mitigate the impact on your academic standing."
        ],
        "resourceCards": [
          "Q: Should I include personal details about the unforeseen circumstances?\nA: Keep it brief; sharing extensive details can seem like oversharing or making excuses.",
          "Q: Do I need to ask for an extension retroactively?\nA: You can mention if you’d appreciate consideration, but be prepared for the standard late penalty unless policy states otherwise.",
          "Q: What if the professor doesn’t respond?\nA: They may note your apology and penalty internally. If you need confirmation, wait a few days before politely following up."
        ]
      },
      {
        "id": "professional-thank-you",
        "title": "Professional Thank-You Note",
        "order": 13,
        "category": "professional",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for a professional thank-you email",
            "example": "Thank You - Informational Interview Follow-Up"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Thank-you email expressing appreciation and highlighting key takeaways",
            "example": "Hello [Name],\n\nThank you for taking the time to speak with me about your experience in the marketing industry. Your insights on campaign analytics and brand storytelling were incredibly valuable. I look forward to staying in touch and applying your advice in my next project.\n\nBest,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 2,
          "scale": 5,
          "factors": [
            "Timeliness",
            "Specificity in gratitude",
            "Maintaining a professional relationship"
          ],
          "baseCompletionPoints": 15,
          "bonusMultiplier": 1.1
        },
        "requiredEloRating": 200,
        "commonMistakes": [
          {
            "id": "m13-1",
            "description": "Being vague or generic about what you are thanking them for"
          },
          {
            "id": "m13-2",
            "description": "Waiting too long (more than a week) to send your thanks"
          },
          {
            "id": "m13-3",
            "description": "Forgetting to mention how their advice or help benefited you"
          },
          {
            "id": "m13-4",
            "description": "Not leaving the door open for future communication"
          }
        ],
        "scenario": {
          "context": "You had an informational interview or meeting with a professional who shared career advice. You want to express genuine appreciation and keep the connection warm.",
          "requirements": [
            "Reference the conversation topic or advice given",
            "Express how their help impacted you",
            "Leave an opening for future dialogue"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of personalization in the note",
            "Evaluation of how well you articulate what you learned",
            "Assessment of politeness and invitation for continued connection"
          ],
          "checklistItems": [
            {
              "id": "c13-1",
              "description": "Mentioned the specific meeting or call context"
            },
            {
              "id": "c13-2",
              "description": "Expressed tangible gratitude or takeaway"
            },
            {
              "id": "c13-3",
              "description": "Kept the tone warm yet professional"
            },
            {
              "id": "c13-4",
              "description": "Invited future communication (lightly)"
            },
            {
              "id": "c13-5",
              "description": "Sent in a timely manner (within 2-3 days)"
            }
          ]
        },
        "contentCards": [
          "Send your thank-you note promptly, ideally within 48 hours",
          "Be specific about what you found valuable",
          "Keep it short and to the point",
          "Reiterate your appreciation",
          "Sign off professionally",
          "Mention any next steps or follow-up you plan to take",
          "Avoid overly formal language; a warm tone is helpful",
          "Double-check for typos and grammar",
          "A thoughtful thank-you can strengthen professional relationships",
          "Stay connected on LinkedIn or via email if they’re open to it"
        ],
        "resourceCards": [
          "Q: Should I offer anything in return for their help?\nA: If relevant, you can offer assistance or resources that might be valuable to them, but don’t force it.",
          "Q: Can I send a handwritten note?\nA: If you have their mailing address and believe it’s appropriate, a handwritten note can be a nice personal touch. Otherwise, email is perfectly acceptable.",
          "Q: How do I ensure I keep the relationship warm?\nA: Occasionally share updates about how their advice has helped you or send relevant articles/opportunities you think they’d appreciate."
        ]
      },
      {
        "id": "graduate-program-intent",
        "title": "Letter of Intent for Graduate Program",
        "order": 14,
        "category": "academic",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for your letter of intent email",
            "example": "Application Letter of Intent - MS in Computer Science"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content summarizing academic background and motivation for the graduate program",
            "example": "Dear Graduate Admissions Committee,\n\nI am writing to express my strong interest in the MS in Computer Science program at XYZ University. My background includes research in machine learning at ABC Lab, and I am eager to further explore this field under the guidance of your esteemed faculty.\n\nThank you for considering my application.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 4,
          "scale": 5,
          "factors": [
            "Academic motivations",
            "Highlighting research interests",
            "Formal admissions process"
          ],
          "baseCompletionPoints": 30,
          "bonusMultiplier": 1.4
        },
        "requiredEloRating": 140,
        "commonMistakes": [
          {
            "id": "m14-1",
            "description": "Failing to tailor the intent letter to the specific program"
          },
          {
            "id": "m14-2",
            "description": "Not showcasing your relevant background or achievements"
          },
          {
            "id": "m14-3",
            "description": "Being too generic about your research interests"
          },
          {
            "id": "m14-4",
            "description": "Omitting your future career or research goals"
          }
        ],
        "scenario": {
          "context": "You are applying for a master's program and need to provide a letter of intent highlighting your motivation, research interests, and fit with the program.",
          "requirements": [
            "Highlight key academic experiences",
            "Explain your research interests or focus",
            "Show alignment with the program’s strengths"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of how well you connect your background to the program’s offerings",
            "Evaluation of clarity and focus in research/academic goals",
            "Assessment of professional and enthusiastic tone"
          ],
          "checklistItems": [
            {
              "id": "c14-1",
              "description": "Stated clear academic and research interests"
            },
            {
              "id": "c14-2",
              "description": "Referenced specific elements of the program (faculty, labs, resources)"
            },
            {
              "id": "c14-3",
              "description": "Highlighted relevant past experiences or achievements"
            },
            {
              "id": "c14-4",
              "description": "Expressed future career or research goals"
            },
            {
              "id": "c14-5",
              "description": "Maintained a formal, respectful tone"
            }
          ]
        },
        "contentCards": [
          "Research the specific faculty and labs associated with the program",
          "Show how your background complements the program’s focus",
          "Emphasize your unique qualifications and experiences",
          "Keep language formal but engaging",
          "Be concise—admissions committees read many letters",
          "Highlight any publications, presentations, or notable projects",
          "Mention your long-term career or research trajectory",
          "Double-check for grammar, clarity, and specificity",
          "Ensure your statement aligns with the program’s mission or strengths",
          "A strong letter of intent can distinguish you in a competitive admissions pool"
        ],
        "resourceCards": [
          "Q: Should I mention potential faculty I’d like to work with?\nA: Yes, if you’ve researched their work and can articulate why you’d be a good fit. Be specific in referencing their lab or research areas.",
          "Q: How long should my letter of intent be?\nA: Typically 1-2 pages (or the word limit set by the program). Focus on quality over length.",
          "Q: Is it okay to mention personal motivations?\nA: Yes, but keep it relevant and professional. Show how your personal drive connects to your academic goals."
        ]
      },
      {
        "id": "grant-request",
        "title": "Academic Grant Funding Request",
        "order": 15,
        "category": "professional",
        "submissionTemplate": {
          "subjectLine": {
            "type": "string",
            "required": True,
            "maxLength": 100,
            "description": "Subject line for requesting research grant funding",
            "example": "Grant Funding Request - AI Ethics Research Proposal"
          },
          "body": {
            "type": "string",
            "required": True,
            "maxLength": 2000,
            "description": "Email content outlining your project proposal, funding needs, and potential impact",
            "example": "Dear Grants Committee,\n\nI am reaching out to request funding for my AI Ethics Research Proposal, which aims to investigate bias detection methods in machine learning models. Enclosed is a brief proposal detailing objectives, methodology, and the projected timeline. The requested funds will primarily cover data acquisition and computational resources.\n\nThank you for your consideration.\n\nSincerely,\n[Your Name]"
          }
        },
        "difficulty": {
          "score": 5,
          "scale": 5,
          "factors": [
            "Detailed project outline",
            "Financial justification",
            "Formal committee review process"
          ],
          "baseCompletionPoints": 35,
          "bonusMultiplier": 1.5
        },
        "requiredEloRating": 220,
        "commonMistakes": [
          {
            "id": "m15-1",
            "description": "Not providing a clear budget breakdown or justification"
          },
          {
            "id": "m15-2",
            "description": "Being vague about the research methodology or impact"
          },
          {
            "id": "m15-3",
            "description": "Failing to follow the specific grant application guidelines"
          },
          {
            "id": "m15-4",
            "description": "Overlooking required attachments or references"
          }
        ],
        "scenario": {
          "context": "You are seeking a research grant for a graduate-level or professional project on AI ethics. You need to clearly present your proposal, justify the budget, and attach supporting documents.",
          "requirements": [
            "Include a concise project description",
            "Outline the budget and justification for requested funds",
            "Reference the potential impact or deliverables of your research"
          ]
        },
        "rubric": {
          "aiFeedbackPoints": [
            "Analysis of clarity in describing project objectives and methods",
            "Evaluation of budget details and justification",
            "Assessment of alignment with grant committee expectations"
          ],
          "checklistItems": [
            {
              "id": "c15-1",
              "description": "Provided a succinct project overview (objectives, significance)"
            },
            {
              "id": "c15-2",
              "description": "Included an itemized budget or at least a cost breakdown"
            },
            {
              "id": "c15-3",
              "description": "Justified the need for each budget item"
            },
            {
              "id": "c15-4",
              "description": "Mentioned expected outcomes or deliverables"
            },
            {
              "id": "c15-5",
              "description": "Adhered to any stated grant proposal guidelines"
            }
          ]
        },
        "contentCards": [
          "Thoroughly review the grant’s requirements before submission",
          "Demonstrate how your research fits with the funding organization's goals",
          "Showcase the potential academic or social impact of your work",
          "Provide a realistic, detailed budget breakdown",
          "Ensure your proposal is succinct and well-structured",
          "Include any preliminary data or references if available",
          "Highlight your qualifications or relevant experience",
          "Address possible challenges or limitations",
          "Proofread for professionalism and clarity",
          "A well-organized, compelling request increases your chances of securing funding."
        ],
        "resourceCards": [
          "Q: How detailed should my budget be?\nA: Include line items for major expenses (e.g., software licenses, data fees, equipment) and show how each item contributes to the project goals.",
          "Q: Can I request more funding later if needed?\nA: This depends on the grant’s rules. Typically, you should request the full amount needed upfront or provide clear stages for additional funding.",
          "Q: What if my proposal is declined?\nA: Politely ask for feedback. You can often revise and resubmit for future calls or apply to other funding sources."
        ]
      }
    ],
    "metadata": {
      "version": "1.0",
      "lastUpdated": "2025-02-19",
      "totalModules": 15,
      "estimatedCompletionTime": "2.5 hours",
      "eloCalculation": {
        "description": "Algorithm for calculating Elo points gained from completion",
        "formula": "baseCompletionPoints * (difficultyScore/5) * bonusMultiplier * (checklistItemsCompleted/totalChecklistItems)",
        "bonusConditions": [
          {
            "condition": "All checklist items completed",
            "multiplier": 1.5
          },
          {
            "condition": "Completed within estimatedTime",
            "multiplier": 1.2
          },
          {
            "condition": "First attempt success",
            "multiplier": 1.3
          }
        ],
        "penaltyConditions": [
          {
            "condition": "Multiple attempts required",
            "multiplier": 0.8
          },
          {
            "condition": "Missing crucial checklist items",
            "multiplier": 0.7
          }
        ]
      }
    }
  }
}
